In [28]:
# | default_exp client

In [29]:
# | hide
from nbdev.showdoc import *

In [30]:
# | exporti
from __future__ import annotations

import os 

from abc import ABC, abstractmethod, abstractproperty
from dataclasses import dataclass, field
from pprint import pprint
import httpx

from dateutil.parser import parse as dtu_parse
import re


In [31]:
#| export
class Auth(ABC):
    """Abstract base class handles authentication.."""

    @property
    @abstractmethod
    def auth_header(self) -> str:
        ...

    @auth_header.setter
    @abstractmethod
    def auth_header(self, val: str):
        ...

    def generate_auth_header(self):
        ...
    

In [32]:
# | export
@dataclass
class ResponseGetData:
    """class for returning data from any route"""

    status: int
    response: str
    auth: any = field(repr=False, default=None)
    is_success: bool = False

    def __post_init__(self):
        self.is_success = True if self.status >= 200 and self.status <= 399 else False

    @classmethod
    def _from_httpx_response(cls, res: httpx.Response, auth=None):
        rgd = cls(status=res.status_code, response=res.json(), auth=auth)

        return rgd

In [33]:
show_doc(ResponseGetData)


---

[source](https://github.com/jaewilson07/gdoc_sync/blob/main/gdoc_sync/client.py#L35){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData

>      ResponseGetData (status:int, response:str, auth:<built-
>                       infunctionany>=None, is_success:bool=False)

class for returning data from any route

In [34]:
ResponseGetData(status=200, response="hello world")

ResponseGetData(status=200, response='hello world', is_success=True)

In [35]:
# | export
async def get_data(
    url: str,
    method: str,
    auth: any,
    headers: dict = None,
    params: dict = None,
    debug_api: bool = False,
    body=None,
    session: httpx.AsyncClient = None,
) -> ResponseGetData:
    """wrapper for httpx Request library, always use with jiralibrary class"""

    is_close_session = False
    method = method.lower()

    if not headers:
        headers = {"Accept": "application/json"}

    if auth:
        headers = {**headers, **auth.generate_auth_header()}

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True

    if debug_api:
        pprint({"url": url, "headers": headers, "params": params, "body": body})

    if method == 'get':
        res = await getattr(session, method)( url=url, headers=headers, params=params )
    else: 
        res = await getattr(session, method)( url=url, headers=headers, params=params, data=body)

    if is_close_session:
        await session.aclose()

    return ResponseGetData._from_httpx_response(res)

In [36]:
# | export
async def looper(
    url,
    auth: ja.JiraAuth,
    arr_fn,
    params : dict = None,
    offset=0,
    limit=50,
    debug_loop: bool = False,
    debug_api: bool = False,
    method="GET",
    **kwargs
):
    final_array = []
    keep_looping = True

    while keep_looping:
        if debug_loop:
            print({"startAt": offset, "maxResults": limit})
        
        new_params = params.copy() if params else {}

        new_params = {** new_params, "startAt": offset, "maxResults": limit}

        res = await get_data(
            auth=auth,
            url=url,
            method=method,
            params=new_params,
            debug_api=debug_api,
            **kwargs
        )

        new_array = arr_fn(res)

        if not new_array or len(new_array) == 0:
            keep_looping = False

        final_array += new_array
        offset += limit
    
    res.response = final_array

    return res

# Utility Functions

In [37]:
#| export
def get_date(datefield) -> datetime.datetime:
    """converts string date to datetime object"""
    return dtu_parse(datefield) if datefield else None

In [38]:
#| export
def upsert_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()


def clean_url_name(path_name):
    valid_chars = r"[^a-zA-Z0-9_]"

    return re.sub(valid_chars, "", path_name)


def generate_file_name(raw_name):
    return clean_url_name(convert_to_snake(raw_name))

In [39]:
# | hide
import nbdev

nbdev.nbdev_export()